In [3]:
import numpy as np

# Build the coefficient matrix A and right-hand side vector b
def build_system(n):
    h = 1 / (n + 1)
    N = n * n  
    A = np.zeros((N, N))
    b = np.full(N, h**2) 
    
    # Fill A with finite difference
    for i in range(N):
        A[i, i] = 4 + h**2  # Main diagonal
        if i % n != 0:  # Left 
            A[i, i - 1] = -1
        if (i + 1) % n != 0:  # Right
            A[i, i + 1] = -1
        if i >= n:  # Top 
            A[i, i - n] = -1
        if i < N - n:  # Bottom 
            A[i, i + n] = -1
    
    return A, b

# Conjugate Gradient Method
def conjugate_gradient(A, b, tol=1e-8, max_iter=1000):
    # Initialization
    x = np.zeros_like(b)
    r = b - np.dot(A, x)
    p = r.copy()
    rs_old = np.dot(r, r)
    
    residuals = []  
    # Implementation of the algorithm
    for i in range(max_iter):
        Ap = np.dot(A, p)
        alpha = rs_old / np.dot(p, Ap)
        x += alpha * p
        r -= alpha * Ap
        rs_new = np.dot(r, r)
        residuals.append(np.sqrt(rs_new))
        if np.sqrt(rs_new) < tol:
            break
        p = r + (rs_new / rs_old) * p
        rs_old = rs_new
    
    i += 1
    
    return i, residuals

# Jacobi Preconditioner
def jacobi_preconditioner(A):
    
    return np.diag(1 / np.diag(A))

# SSOR Preconditioner with omega = 1
def ssor_preconditioner(A, omega = 1):
    # Decomposition
    D = np.diag(np.diag(A)) 
    L = np.tril(A, -1)       
    U = np.triu(A, 1)        

    # Compute the SSOR preconditioner matrix
    D_plus_omega_L_inv = np.linalg.inv(D + omega * L)
    D_plus_omega_U_inv = np.linalg.inv(D + omega * U)
    M = np.dot(np.dot(D_plus_omega_L_inv, D), D_plus_omega_U_inv) * (2 - omega)

    return M


# Preconditioned Conjugate Gradient Method
def preconditioned_conjugate_gradient(A, b, M_inv, tol=1e-8, max_iter=1000):
    x = np.zeros(b.shape)
    r = b - np.dot(A, x)
    z = np.dot(M_inv, r)
    p = z.copy()
    rs_old = np.dot(r, z)

    residuals = []      
    #Implementation of the Algorithm
    for i in range(max_iter):
        Ap = np.dot(A, p)
        alpha = rs_old / np.dot(p, Ap)
        x += alpha * p
        r -= alpha * Ap
        z = np.dot(M_inv, r)
        rs_new = np.dot(r, z)
        residuals.append(np.sqrt(rs_new))
        if np.sqrt(rs_new) < tol:
            break
        p = z + (rs_new / rs_old) * p
        rs_old = rs_new
    
    i += 1
    
    return i, residuals

In [4]:
# test for n = 8, 16, 32
n_values = [8, 16, 32]
results = []

for n in n_values:
    A, b = build_system(n)
    
    # Conjugate Gradient
    cg_iterations, _ = conjugate_gradient(A, b)
    
    # Jacobi PCG
    M_jacobi = jacobi_preconditioner(A)
    jacobi_iterations, _ = preconditioned_conjugate_gradient(A, b, M_jacobi)
    
    # SSOR PCG
    M_ssor = ssor_preconditioner(A)
    ssor_iterations, _ = preconditioned_conjugate_gradient(A, b, M_ssor)
    
    # Collect results
    results.append({
        "n": n,
        "CG Iterations": cg_iterations,
        "PCG Jacobi Iterations": jacobi_iterations,
        "PCG SSOR Iterations": ssor_iterations
    })

# Display results
print("\nComparison of Iterations for Each Method:")
for result in results:
    print(f"n = {result['n']}: CG = {result['CG Iterations']}, PCG (Jacobi) = {result['PCG Jacobi Iterations']}, PCG (SSOR) = {result['PCG SSOR Iterations']}")


0.0034730953470973854
0.0016855917267914657
0.0008462593840523033
0.00023804339418451894
3.228614604932628e-05
2.1700091790263587e-06
2.0797740710389087e-07
5.58783027594462e-09
9.869287624829178e-11
5.599734459920357e-35
0.0031149509092507585
0.00024349039779109527
4.510066969419287e-06
1.71895194832219e-07
3.738780504645556e-09
1.1498038906509605e-10
1.4482477981422511e-11
3.220033641901553e-13
1.415831187880543e-15
5.869503709223646e-18
0.002606977318912745
0.0018970850579315504
0.0016548464667311348
0.0011806523735193063
0.00079447426753812
0.0005304571768337488
0.0003092395587018917
0.00016941188117486312
6.994765115198784e-05
2.0355024049633434e-05
3.898924264941201e-06
1.2139894434761258e-06
7.867398171963086e-07
2.1152579695073822e-07
5.984985125988719e-08
2.1030962679392467e-08
5.29739683191936e-09
1.3287615512201224e-09
2.0298601517241786e-10
2.757490198850755e-11
2.2481987270636857e-12
1.2917094997141288e-13
1.3765932747855674e-14
2.9944064890756344e-15
1.8376338576231703e-1